In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, regularizers
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler

In [91]:
#Setup dataset

data = pd.read_csv("./data/data.csv")

labels = data['order']
data['key'] = data['key']/11
data['loudness'] = data['loudness']/-32 #Is wrong (MinMaxScaler)
data['tempo'] = data['tempo']/250
data['time_signature'] = data['time_signature']/5

columns_to_remove = ['Unnamed: 0', 'id', 'duration_ms', 'track_title', 'album_title', 'album_artist', 'track_number', 'total_tracks']#, 'order']
data.drop(columns = columns_to_remove, axis = 1, inplace = True)

training_set, test_set, training_labels, test_labels = train_test_split(data, labels, test_size=0.25, random_state=42)

training_set

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,order
3790,0.776,0.710,0.727273,0.192688,1,0.0464,0.00224,0.002270,0.2100,0.5690,0.479912,0.8,0.100000
156,0.449,0.198,0.181818,0.461813,1,0.0343,0.49000,0.917000,0.1060,0.0399,0.527464,0.8,0.516129
4116,0.725,0.632,0.545455,0.168656,0,0.0364,0.03430,0.100000,0.1590,0.6510,0.488156,0.8,0.750000
1720,0.660,0.817,1.000000,0.195906,0,0.0521,0.16200,0.000000,0.0667,0.7140,0.543768,0.8,0.846154
615,0.737,0.757,0.454545,0.278250,0,0.0993,0.01990,0.845000,0.3950,0.8800,0.512004,0.8,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,0.399,0.522,0.636364,0.226969,1,0.0325,0.09640,0.000073,0.4310,0.2900,0.648544,0.6,0.720000
466,0.833,0.877,0.090909,0.107531,0,0.2160,0.00589,0.121000,0.6690,0.0380,0.511916,0.8,0.840000
3092,0.456,0.848,0.636364,0.204906,1,0.0407,0.00149,0.192000,0.2760,0.0870,0.519880,0.8,0.615385
3772,0.416,0.560,0.545455,0.232687,0,0.0389,0.00562,0.001910,0.3010,0.4710,0.414588,0.8,0.636364


In [ ]:
# Old autoencoder

#  latent_dim = 1

# class Autoencoder(Model):
#   def __init__(self, latent_dim):
#     super(Autoencoder, self).__init__()
#     self.latent_dim = latent_dim   
#     self.encoder = tf.keras.Sequential([
#       layers.Flatten(),
#       layers.Dense(latent_dim, activation='relu'),
#     ])
#     self.decoder = tf.keras.Sequential([
#       layers.Dense(12, activation='sigmoid'),
#       #layers.Reshape((12))
#     ])

#   def call(self, x):
#     encoded = self.encoder(x)
#     decoded = self.decoder(encoded)
#     return decoded
  
# autoencoder = Autoencoder(latent_dim)

# autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [122]:
#Setup autoencoder

input_track = tf.keras.Input(shape=(13,))

encoded = layers.Dense(10, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(10, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(5, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(5, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(1, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)
encoded = layers.Dense(1, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)

decoded = layers.Dense(3, activation='relu')(encoded)
decoded = layers.Dense(3, activation='relu')(encoded)
decoded = layers.Dense(5, activation='relu')(encoded)
decoded = layers.Dense(5, activation='relu')(encoded)
decoded = layers.Dense(10, activation='relu')(encoded)
decoded = layers.Dense(10, activation='relu')(encoded)
decoded = layers.Dense(13, activation='sigmoid')(encoded)
decoded = layers.Dense(13, activation='sigmoid')(encoded)

autoencoder = tf.keras.Model(input_track, decoded)

encoder = tf.keras.Model(input_track, encoded)

encoded_input = tf.keras.Input(shape=(1,))
decoder_layer = autoencoder.layers[-1]
decoder = tf.keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [123]:
autoencoder.fit(training_set, training_set,
                epochs=200,
                shuffle=True,
                validation_split = 0.2)

Epoch 1/200
93/93 [==============================] - 0s 3ms/step - loss: 0.1059 - val_loss: 0.1015
Epoch 2/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0987 - val_loss: 0.0947
Epoch 3/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0905 - val_loss: 0.0822
Epoch 4/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0775 - val_loss: 0.0708
Epoch 5/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0698 - val_loss: 0.0654
Epoch 6/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0661 - val_loss: 0.0627
Epoch 7/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0641 - val_loss: 0.0611
Epoch 8/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0628 - val_loss: 0.0600
Epoch 9/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0618 - val_loss: 0.0590
Epoch 10/200
93/93 [==============================] - 0s 2ms/step - loss: 0.0609 - val_loss: 0.0581
Epoch 11/

In [124]:
encoded_tracks = encoder.predict(test_set)
decoded_tracks = decoder.predict(encoded_tracks)

print(encoded_tracks[0])
print(decoded_tracks[0])
print(test_set.iloc[0])
for i in range(13):
    print(decoded_tracks[0][i]-test_set.iloc[0, i])

[0.8479278]
[0.58206105 0.6409404  0.5036648  0.24311519 0.99122834 0.1058946
 0.08465415 0.20476812 0.22089046 0.38911757 0.4800329  0.784551
 0.49619544]
danceability        0.389000
energy              0.944000
key                 0.363636
loudness            0.187219
mode                1.000000
speechiness         0.105000
acousticness        0.010000
instrumentalness    0.001540
liveness            0.068800
valence             0.187000
tempo               0.579692
time_signature      0.800000
order               0.440000
Name: 3135, dtype: float64
0.19306105232238768
-0.3030595722198486
0.14002842794765125
0.05589643669128419
-0.008771657943725586
0.0008945953845977822
0.07465415239334107
0.20322812124252318
0.15209046239852905
0.20211756873130798
-0.09965910896492003
-0.01544897556304936
0.056195435523986814
